In [1]:
import pandas as pd
import os
import glob

In [9]:
images = os.listdir("images/train")
images = [i.replace(".jpg", "") for i in images]
pd_img = pd.DataFrame({'id':images, 'is_downloaded': 1})

train_file = pd.read_csv("input/Google-Landmarks-Dataset/train.csv")
train_file = train_file.merge(pd_img, on='id', how='left')
train_file['is_downloaded'].fillna(0, inplace=True)
train_file.head()


,id,url,landmark_id,is_downloaded
0,97c0a12e07ae8dd5,http://lh4.ggpht.com/-f8xYA5l4apw/RSziSQVaABI/...,6347,1.0
1,650c989dd3493748,https://lh5.googleusercontent.com/-PUnMrX7oOyA...,12519,1.0
2,05e63ca9b2cde1f4,http://mw2.google.com/mw-panoramio/photos/medi...,264,0.0
3,08672eddcb2b7c93,http://lh3.ggpht.com/-9fgSxDYwhHA/SMvGEoltKTI/...,13287,1.0
4,fc49cb32ef7f1e89,http://lh6.ggpht.com/-UGAXxvPbr98/S-jGZbyMIPI/...,4018,1.0


# How many images per each landmark

In [22]:
img_landmark = train_file.landmark_id.value_counts().reset_index()
img_landmark.columns = ["landmark_id", "N"]
img_landmark.describe()

,N
count,14948.000000
mean,81.952703
std,740.289706
min,1.000000
25%,6.000000
50%,13.000000
75%,44.000000
max,49531.000000


- There are about 15k landmarks on the file
2. There landmarks where there are only 1 images! These are useless for training our model. 

Let's just get landmarks that have over 1k images and under 3k

# Get the landmarks with enough (but not too much images)

In [24]:
img_landmark_filter = img_landmark[(img_landmark.N >= 1000) & (img_landmark.N <= 3000)]
img_landmark_filter.describe()

,N
count,84.000000
mean,1392.202381
std,345.414358
min,1004.000000
25%,1121.000000
50%,1305.000000
75%,1623.250000
max,2541.000000


Ok, we now have about 84 landmarks. Using our machine spec, that would be enough to make an image retrieval system.

In [26]:
train_file_filter = train_file.merge(img_landmark_filter, on='landmark_id', how='inner')
train_file_filter.head()

,id,url,landmark_id,is_downloaded,N
0,97c0a12e07ae8dd5,http://lh4.ggpht.com/-f8xYA5l4apw/RSziSQVaABI/...,6347,1.0,1052
1,61cbf8229c5d5242,http://lh4.ggpht.com/-QXV3dWBZU4U/Th8_jFTcoLI/...,6347,1.0,1052
2,e07746b5f49a3a40,https://lh4.googleusercontent.com/---Es7LxSNHI...,6347,1.0,1052
3,02eac09035f17862,http://lh4.ggpht.com/-98979yuMKDE/ScyrA4Q0egI/...,6347,1.0,1052
4,e96691afeabc7ca4,https://lh4.googleusercontent.com/-EloLlWjh6hg...,6347,1.0,1052


# Get the list of downloaded, selected landmark images, delete the other images

In [36]:
count_deleted = 0

In [ ]:
train_file_filter_downloaded = train_file_filter[train_file_filter.is_downloaded == 1]
img_chosen = [i + ".jpg" for i in train_file_filter_downloaded.id.values]
images = os.listdir("images/train")

for i in images:
    if i not in img_chosen:
        #Not in the selected list, delete downloaded images
        os.remove("images/train/" + i)
        count_deleted += 1
        